In this notebook, I will...
   - calculate baseline error
   - run random forest regressors on abundance 
   - load regional dfs, impute, add island col as ID, rbind all 5 dfs 
   - predict out for each group 

#### Scoring models

We evaluated the performance of the abundance models based on mean average error (accuracy) and mean absolute percent error (MAPE).

## Import modules

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, confusion_matrix, auc
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import *
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
import pickle
import requests
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [2]:
os.chdir('C:/Users/linds/OneDrive/Documents/samoa_corals_data')

## Import data

In [3]:
coral_types=['plate']
target_types=['binary','percent']
df = dict()

for i in range(0,len(coral_types)):
    for j in range(0,len(target_types)):
        df[str(coral_types[i])+'_'+str(target_types[j])]=pd.read_csv(str(coral_types[i])+'_'+str(target_types[j])+'.csv')
        del df[str(coral_types[i])+'_'+str(target_types[j])]['Unnamed: 0'] # artifact indexing column
# Access the data as, e.g., df['scler_percent']

# Scler (all corals)

### Train/test split, baseline error

The baseline is the estimate I would get if I simply predicted the average abundance across all cells. If I can improve upon this by using my model, then my approach is valid.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df['plate_percent'].drop(['Plt_L_R','lat','lon','ID'], axis=1), 
                                                    df['plate_percent']['Plt_L_R'], 
                                                    test_size = 0.3, random_state = 30)


print('Plate Training Features Shape:', X_train.shape)
print('Plate Training Labels Shape:', y_train.shape)
print('Plate Testing Features Shape:', X_test.shape)
print('Plate Testing Labels Shape:', y_test.shape)

# The baseline predictions are the averages

baseline_preds = np.array([y_train.mean()] * len(y_train))
baseline_errors = abs(baseline_preds - y_train)
print('Baseline prediction error: ', round(np.mean(baseline_errors), 2))

Plate Training Features Shape: (1372, 10)
Plate Training Labels Shape: (1372,)
Plate Testing Features Shape: (588, 10)
Plate Testing Labels Shape: (588,)
Baseline prediction error:  9.3


### Train random forest regressor

In [5]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 30)

# Train the model on training data
rf.fit(X_train, y_train)

# Predicting to the test data
predictions = rf.predict(X_test)

# Calculate the absolute errors
errors = abs(predictions - y_test)

# Print out the mean absolute error (mae)
print('Mean Absolute Error: ', round(np.mean(errors), 2))

Mean Absolute Error:  9.03


### Tune the model

In [6]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
bootstrap = [True, False] # Create the random grid

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'bootstrap': bootstrap}

print(random_grid)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()

# Random search of parameters, using 10 fold cross validation, 
# Search across n_iter * cv different combinations, and use all available cores
rf_best = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 40, cv = 10, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf_best.fit(X_train, y_train)
print(rf_best.best_params_)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'bootstrap': [True, False]}
Fitting 10 folds for each of 40 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:  5.5min finished


{'n_estimators': 1000, 'max_features': 'sqrt', 'bootstrap': True}


In [7]:
#Predicting on the test set
predictions = rf_best.predict(X_test)

# Calculate the absolute errors
errors = abs(predictions - y_test)

# Print out the mean absolute error (mae)
print('Mean Absolute Error: ', round(np.mean(errors), 2))

Mean Absolute Error:  8.77


In [6]:
# Retrain with the optimal parameters
rf = RandomForestRegressor(n_estimators = 1000, max_features = 'sqrt', bootstrap = True, random_state = 30)

# Train the model on training data
rf.fit(X_train, y_train)

# Predicting to the test data
predictions = rf.predict(X_test)

# Calculate the absolute errors
errors = abs(predictions - y_test)

# Print out the mean absolute error (mae)
print('Mean Absolute Error: ', round(np.mean(errors), 2))

# Get numerical feature importances
feature_list = list(X_train.columns)
importances = list(rf.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
print(feature_importances)

# Save the test data (actual) and predictions to df for visualizations later
plate_preds = pd.DataFrame(data = {'actual': y_test, 'prediction': predictions})
plate_preds.to_csv('plate_preds.csv', index=False)

Mean Absolute Error:  8.79
[('bty', 0.15), ('rug', 0.13), ('slope', 0.12), ('vill_ed', 0.12), ('asp', 0.11), ('shore_ed', 0.11), ('esi_ed', 0.11), ('hs', 0.07), ('ucur', 0.04), ('vcur', 0.04)]


### Pickle the model

In [18]:
#with open('plate_abundance_model.pkl', 'wb') as fid:
#    pickle.dump(rf, fid, 2)

### Predict out

In [9]:
AS_all = pd.read_csv('AS_all.csv')

# plate: bty, hs, rug, slope, asp, shore_ed, vill_ed, esi_ed, ucur, vcur
plate_pred_out = AS_all[['bty', 'hs', 'rug', 'slope', 'asp', 'shore_ed', 'vill_ed', 'esi_ed', 'ucur', 'vcur']]
plate_predictions = rf_best.predict(plate_pred_out)

print(y_train.mean())
print(plate_predictions.mean())
AS_all['plate_preds'] = plate_predictions
AS_all.to_csv('AS_all.csv', index=False)

4.976822157434403
11.176227688116423
